In [1]:
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import camb
from camb import model, initialpower
import math
from scipy.integrate import simps as simps

In [ ]:
pars = camb.CAMBparams()
pars.set_cosmology(H0 = 72)
results = camb.get_background(pars)

In [ ]:
def alpha_distance(z):
    some code here    
    return D_alpha

In [ ]:
# check this matches results from standard CAMB lum distance function
Hubble_param = 72
omega_m = .3
omega_lambda = .7

z = np.linspace(0,4).reshape(-1,1)

def integrand(z):
    return 1 / math.sqrt( (omega_m * (1 + z_prime)**3 + omega_lambda) )
scipy.integrate.simps(integrand(z), z, axis=0)

In [23]:
Hubble_param = 72
omega_m = .3
omega_lambda = .7

def func( x ):
    return lambda x : 1 / math.sqrt( x ) # issue because sqrt converts array into scalar, use lambda to map values

def func2( x ):
    return (omega_m * (1 + a * x)**3 + omega_lambda)

def integrand( x ):
    return func2( x) #func(func2( x ))

a = np.array([1,2,3])
x = np.linspace(0,4).reshape(-1,1)
simps(integrand(x), x, axis=0)

array([ 49.6002448 , 248.80163197, 716.80514072])

In [ ]:
def lum_distance(z):
    factor = (1 + z) / Hubble_param
    #integral = scipy.integrate.simps(y = integrand(np.linspace(0,4)), x = np.linspace(0,4))
    #D_l = factor * res
    return 0

answer = lum_distance(z)
answer

In [ ]:
# calculate distances for redshifts 0 to 4
z = np.linspace(0,4)
camb_Dl = results.luminosity_distance(z)
camb_Dl